In [ ]:
# Setup & Environment
# ---------------------------
# This cell initializes the environment for our pub figures analysis. It imports all
# necessary libraries, sets up the expanded Arcadia color palette and a custom
# Plotly template for consistent, high-quality visualizations, and defines the
# file paths for all inputs and outputs for this new analysis phase.

# --- Standard Library Imports ---
import logging
from pathlib import Path

# --- Plotting Libraries ---
import arcadia_pycolor as apc
import numpy as np

# --- Data Handling ---
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# --- Statistics & Machine Learning ---
from scipy import stats

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] - %(message)s")
logger = logging.getLogger()

# --- Define File Paths ---
BASE_DIR = Path(".").resolve()
ANALYSIS_DIR = BASE_DIR / "analysis_pub_figures"
PLOTS_DIR = BASE_DIR / "plots_pub_figures"

# Create output directories
PLOTS_DIR.mkdir(parents=True, exist_ok=True)

# Input files for this notebook
ALL_VS_ALL_STRUCTURAL_FILE = ANALYSIS_DIR / "all_vs_all_structural_metrics.csv"
SEQ_DIVERSITY_FILE = ANALYSIS_DIR / "og_sequence_diversity_v3.csv"

logging.info("Setup complete. pub figures analysis environment is ready.")

In [ ]:
# Setup & Configuration (Completed)
# -----------------------------------------
# This cell configures the notebook's environment. It sets up logging, applies
# the official Arcadia Science Plotly template, and defines all necessary file paths.

# --- Setup Logging ---
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] - %(message)s")
logger = logging.getLogger()

# --- Setup Arcadia Plotly Template ---
apc.plotly.setup()
logging.info("Default Plotly theme set to 'arcadia' via arcadia-pycolor.")

# --- Define Directory Paths ---
# Assumes the notebook is in the root of the project directory.
BASE_DIR = Path(".").resolve()
ANALYSIS_DIR = BASE_DIR / "analysis_v3"
DATA_DIR = BASE_DIR / "data"
PLOTS_DIR = BASE_DIR / "publication_figures"

# Create the output directory for the final figures
PLOTS_DIR.mkdir(parents=True, exist_ok=True)
logging.info(f"Publication figures will be saved to: {PLOTS_DIR}")

# --- Define Input File Paths ---
# These are the final, processed data files from your analysis notebook.
ALL_VS_ALL_STRUCTURAL_FILE = ANALYSIS_DIR / "all_vs_all_structural_metrics.csv"
SEQ_DIVERSITY_FILE = ANALYSIS_DIR / "og_sequence_diversity_v3.csv"
PROTEOME_DB_FILE = (
    DATA_DIR / "proteome_database_v3.5.csv"
)  # <-- CORRECTED: Path is now defined here
CONSERVATION_DATA_FILE = ANALYSIS_DIR / "generated_data" / "df_per_column_conservation.csv"

logging.info("Environment setup is complete. All paths are defined.")

In [ ]:
# Load and Prepare Master DataFrame
# -----------------------------------------
# This cell loads the pre-calculated sequence and structural diversity data,
# then merges them into a single master DataFrame for analysis. It combines
# the logic from your original Cells 2 and 3 to make the data loading
# process more robust.

# Initialize df_master as an empty DataFrame to prevent NameError in subsequent cells
df_master = pd.DataFrame()

try:
    # --- 1. Load & Aggregate Structural Diversity Data ---
    logging.info(f"Loading all-vs-all structural metrics from: {ALL_VS_ALL_STRUCTURAL_FILE}")
    df_struct_raw = pd.read_csv(ALL_VS_ALL_STRUCTURAL_FILE)
    logging.info(f"Loaded {len(df_struct_raw)} pairwise comparisons.")

    df_struct_raw.dropna(subset=["TMscore_1", "TMscore_2", "RMSD"], inplace=True)
    logging.info(f"{len(df_struct_raw)} pairs remain after dropping errors.")

    logging.info("Aggregating pairwise metrics to OG-level structural diversity stats...")
    aggregations = {"TMscore_1": ["mean", "median", "std"], "RMSD": ["mean", "median", "std"]}
    df_struct_diversity = df_struct_raw.groupby("OG_ID").agg(aggregations)

    # Flatten the multi-level column index and rename for clarity
    df_struct_diversity.columns = [
        "_".join(col).strip() for col in df_struct_diversity.columns.values
    ]
    df_struct_diversity = df_struct_diversity.rename(
        columns={
            "TMscore_1_mean": "Mean_TMscore",
            "TMscore_1_median": "Median_TMscore",
            "TMscore_1_std": "StdDev_TMscore",
            "RMSD_mean": "Mean_RMSD",
            "RMSD_median": "Median_RMSD",
            "RMSD_std": "StdDev_RMSD",
        }
    )
    df_struct_diversity.reset_index(inplace=True)
    logging.info(
        f"Successfully calculated structural diversity metrics for {len(df_struct_diversity)} OGs."
    )

    # --- 2. Load Sequence Diversity Data ---
    logging.info(f"Loading sequence diversity data from: {SEQ_DIVERSITY_FILE}")
    df_seq_diversity = pd.read_csv(SEQ_DIVERSITY_FILE)
    logging.info(f"Loaded sequence diversity metrics for {len(df_seq_diversity)} OGs.")

    # Clean up the OG_ID column to standardize the format for merging
    logging.info("Cleaning OG_ID column in sequence diversity data to standardize format...")
    df_seq_diversity["OG_ID"] = df_seq_diversity["OG_ID"].str.replace(
        "_len_filtered_final", "", regex=False
    )

    # --- 3. Merge into Master DataFrame ---
    df_master = pd.merge(df_seq_diversity, df_struct_diversity, on="OG_ID", how="inner")
    logging.info(f"Master dataframe created with {len(df_master)} OGs after merging.")

    # --- 4. Final Data Cleaning ---
    key_cols = ["Tree_Hill_Diversity_q1_NormByTips", "StdDev_TMscore", "Mean_TMscore"]
    df_master.dropna(subset=key_cols, inplace=True)
    logging.info(f"{len(df_master)} OGs remain after final cleaning.")

    print("\\nMaster DataFrame created successfully:")
    display(df_master.head())
    print("\\nMaster DataFrame Info:")
    df_master.info()

except FileNotFoundError as e:
    logging.error(
        f"Data file not found: {e}. Please ensure the paths in Cell 1 are correct and the necessary analysis has been run."
    )
except Exception as e:
    logging.error(f"An error occurred while loading or merging data: {e}", exc_info=True)

In [ ]:
# Create Structural Profile Categories
# ---------------------------------------------
# This cell creates the categorical "Structural_Profile" column based on the
# distribution of Mean_TMscore and StdDev_TMscore. This logic is identical
# to the V3 analysis notebook to ensure consistency.

if not df_master.empty:
    try:
        logging.info("Defining structural profile categories...")

        # --- 1. Define Thresholds using Quantiles (25th and 75th percentiles) ---
        mean_tm_low_thresh = df_master["Mean_TMscore"].quantile(0.25)
        mean_tm_high_thresh = df_master["Mean_TMscore"].quantile(0.75)

        stddev_tm_low_thresh = df_master["StdDev_TMscore"].quantile(0.25)
        stddev_tm_high_thresh = df_master["StdDev_TMscore"].quantile(0.75)

        # --- 2. Create Binned Level Columns ---
        conditions_mean = [
            df_master["Mean_TMscore"] < mean_tm_low_thresh,
            df_master["Mean_TMscore"] >= mean_tm_high_thresh,
        ]
        choices_mean = ["Low_Mean_TM", "High_Mean_TM"]
        df_master["Mean_TM_Level"] = np.select(
            conditions_mean, choices_mean, default="Medium_Mean_TM"
        )

        conditions_std = [
            df_master["StdDev_TMscore"] < stddev_tm_low_thresh,
            df_master["StdDev_TMscore"] >= stddev_tm_high_thresh,
        ]
        choices_std = ["Low_StdDev_TM", "High_StdDev_TM"]
        df_master["StdDev_TM_Level"] = np.select(
            conditions_std, choices_std, default="Medium_StdDev_TM"
        )

        # --- 3. Create Descriptive Structural Profile ---
        def assign_structural_profile(row):
            if row["Mean_TM_Level"] == "High_Mean_TM" and row["StdDev_TM_Level"] == "Low_StdDev_TM":
                return "Structurally Rigid"
            elif row["StdDev_TM_Level"] == "High_StdDev_TM":
                return "Structurally Plastic"
            elif (
                row["Mean_TM_Level"] == "Low_Mean_TM" and row["StdDev_TM_Level"] == "Low_StdDev_TM"
            ):
                return "Consistently Dissimilar"
            else:
                return "Intermediate"

        df_master["Structural_Profile"] = df_master.apply(assign_structural_profile, axis=1)

        logging.info("Successfully created 'Structural_Profile' categories.")

        print("Counts per Structural Profile:")
        display(df_master["Structural_Profile"].value_counts())

    except Exception as e:
        logging.error(f"An error occurred while creating structural profiles: {e}", exc_info=True)
else:
    logging.warning("df_master is empty. Skipping profile creation.")

In [ ]:
# Generate Diversity Landscape Plot
# -----------------------------------------
# This cell creates the core visualization for the analysis. It plots sequence
# diversity vs. structural diversity. The points are colored to specifically
# highlight the 'Structurally Rigid' and 'Structurally Plastic' profiles,
# while the other categories are de-emphasized with a neutral color.

if not df_master.empty and "Structural_Profile" in df_master.columns:
    try:
        logging.info("Generating plot of the full diversity landscape with custom highlighting...")

        # --- 1. Define Axes and Marker Size ---
        x_axis_metric = "Tree_Hill_Diversity_q1_NormByTips"
        y_axis_metric = "StdDev_TMscore"

        if "Marker_Size" not in df_master.columns:
            size_bins = [0, 50, 200, 500, 1000, df_master["MSA_N_Seqs"].max() + 1]
            size_labels = ["<50", "50-200", "201-500", "501-1000", ">1000"]
            df_master["OG_Size_Bin"] = pd.cut(
                df_master["MSA_N_Seqs"], bins=size_bins, labels=size_labels, right=False
            )
            size_map = {"<50": 8, "50-200": 12, "201-500": 16, "501-1000": 20, ">1000": 24}
            df_master["Marker_Size"] = df_master["OG_Size_Bin"].map(size_map).fillna(8)

        # --- 2. Define Custom Color Map ---
        # Highlight 'Rigid' and 'Plastic', de-emphasize others with grey.
        profile_color_map = {
            "Structurally Rigid": apc.aegean,
            "Structurally Plastic": apc.rose,
            "Consistently Dissimilar": apc.gray,
            "Intermediate": apc.gray,
        }

        # --- 3. Create the Scatter Plot ---
        fig = px.scatter(
            df_master,
            x=x_axis_metric,
            y=y_axis_metric,
            color="Structural_Profile",
            color_discrete_map=profile_color_map,
            size="Marker_Size",
            hover_name="OG_ID",
            hover_data=["Mean_TMscore", "StdDev_TMscore", "MSA_N_Seqs"],
            title="Sequence vs. Structural Diversity Landscape",
            labels={
                x_axis_metric: "Sequence Diversity (Normalized Hill Diversity)",
                y_axis_metric: "Structural Diversity (StdDev TM-score)",
            },
            # Control legend order
            category_orders={
                "Structural_Profile": [
                    "Structurally Rigid",
                    "Structurally Plastic",
                ]
            },
        )

        # --- 4. Refine Plot Aesthetics ---
        fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color="DarkSlateGrey")))

        # Further de-emphasize the grey points
        for trace in fig.data:
            if trace.name in ["Consistently Dissimilar", "Intermediate"]:
                trace.marker.opacity = 0
                trace.marker.symbol = "circle"

        fig.update_layout(width=900, height=700, legend_title_text="<b>Structural Profile</b>")

        fig.show()

        # --- 5. Export the Figure ---
        plot_filename_base = "fig1.1_diversity_landscape_highlighted"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")
        logging.info(f"Saved highlighted landscape plot to {PLOTS_DIR}")

    except Exception as e:
        logging.error(f"An error occurred while creating the plot: {e}", exc_info=True)
else:
    logging.warning("df_master or 'Structural_Profile' column not found. Skipping plot generation.")

In [ ]:
# Domain Architecture Distribution Plot
# ---------------------------------------------
# This cell loads the domain architecture data, merges it with the master
# dataframe, and generates an overlaid, smoothed histogram (KDE plot) to
# compare the distribution of domain counts across different structural profiles.

import plotly.figure_factory as ff

# --- Check for necessary variables from previous cells ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run the previous cells first."
    )
elif "DATA_DIR" not in locals() or "PLOTS_DIR" not in locals():
    logging.error(
        "Path variables 'DATA_DIR' or 'PLOTS_DIR' are not defined. Please run the setup cell (Cell 2)."
    )
else:
    # --- 1. Load and Prepare Domain Data ---
    # This step is wrapped in a try-except block to handle potential file errors.
    try:
        # Check if the domain data has already been merged
        if "Mean_Num_Domains" not in df_master.columns:
            logging.info("Loading domain information from the main proteome database...")
            proteome_db_path = DATA_DIR / "proteome_database_v3.5.csv"
            proteome_cols = ["OG_ID", "Num_Domains"]

            df_proteome_domains = pd.read_csv(
                proteome_db_path, usecols=proteome_cols, low_memory=False
            )

            df_proteome_domains["Num_Domains"] = pd.to_numeric(
                df_proteome_domains["Num_Domains"], errors="coerce"
            )
            df_proteome_domains.dropna(subset=["OG_ID", "Num_Domains"], inplace=True)

            logging.info("Calculating average number of domains per OG...")
            df_og_domain_avg = (
                df_proteome_domains.groupby("OG_ID")["Num_Domains"].mean().reset_index()
            )
            df_og_domain_avg = df_og_domain_avg.rename(columns={"Num_Domains": "Mean_Num_Domains"})

            # Merge into the master dataframe
            df_master = pd.merge(df_master, df_og_domain_avg, on="OG_ID", how="left")
            logging.info("Successfully merged average domain counts into master dataframe.")
        else:
            logging.info("'Mean_Num_Domains' column already exists in the master dataframe.")

        # --- 2. Generate the Distribution Plot ---
        logging.info("Generating domain architecture distribution plot...")

        # Prepare the data for the three groups
        all_ogs_data = df_master["Mean_Num_Domains"].dropna()
        rigid_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "Mean_Num_Domains"
        ].dropna()
        plastic_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "Mean_Num_Domains"
        ].dropna()

        hist_data = [all_ogs_data, rigid_ogs_data, plastic_ogs_data]
        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]

        # Define colors for the distributions (using a defined color from the palette)
        colors = [apc.slate, apc.aegean, apc.rose]

        # Create the distribution plot with smoothed curves (KDE)
        fig = ff.create_distplot(
            hist_data,
            group_labels,
            bin_size=0.2,
            show_hist=False,  # We only want the smoothed curve
            show_rug=False,
            colors=colors,
        )

        # --- 3. Style and Annotate the Plot ---
        fig.update_layout(
            title="Distribution of Mean Domain Count per OG",
            xaxis_title="Mean Number of Domains",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )

        # Add a vertical line for the mean of all OGs
        mean_all = all_ogs_data.mean()
        fig.add_vline(
            x=mean_all,
            line_width=2,
            line_dash="dash",
            line_color=apc.charcoal,
            annotation_text=f"Mean (All): {mean_all:.2f}",
            annotation_position="top right",
        )

        fig.show()

        # --- 4. Export the Figure ---
        plot_filename_base = "fig3a_domain_dist_plot"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")
        logging.info(f"Saved domain distribution plot to {PLOTS_DIR}")

    except FileNotFoundError:
        logging.error(
            f"Proteome database not found at {proteome_db_path}. Cannot generate domain plot."
        )
    except Exception as e:
        logging.error(f"An error occurred while creating the distribution plot: {e}", exc_info=True)

In [ ]:
# Intrinsic Disorder Distribution Plot
# --------------------------------------------
# This cell applies the same logic as the domain architecture plot to investigate
# the distribution of intrinsic disorder across the different structural profiles.

import plotly.figure_factory as ff

# --- Check for necessary variables from previous cells ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run the previous cells first."
    )
elif "PROTEOME_DB_FILE" not in locals():
    logging.error(
        "Path variable 'PROTEOME_DB_FILE' is not defined. Please run the setup cell (Cell 2)."
    )
else:
    # --- 1. Load and Prepare Intrinsic Disorder Data ---
    try:
        # Check if the disorder data has already been merged
        if "Mean_Percent_Disorder" not in df_master.columns:
            logging.info(
                "Loading intrinsic disorder information from the main proteome database..."
            )
            proteome_cols = ["OG_ID", "Percent_Disorder"]

            df_proteome_disorder = pd.read_csv(
                PROTEOME_DB_FILE, usecols=proteome_cols, low_memory=False
            )

            df_proteome_disorder["Percent_Disorder"] = pd.to_numeric(
                df_proteome_disorder["Percent_Disorder"], errors="coerce"
            )
            df_proteome_disorder.dropna(subset=["OG_ID", "Percent_Disorder"], inplace=True)

            logging.info("Calculating average percent disorder per OG...")
            df_og_disorder_avg = (
                df_proteome_disorder.groupby("OG_ID")["Percent_Disorder"].mean().reset_index()
            )
            df_og_disorder_avg = df_og_disorder_avg.rename(
                columns={"Percent_Disorder": "Mean_Percent_Disorder"}
            )

            # Merge into the master dataframe
            df_master = pd.merge(df_master, df_og_disorder_avg, on="OG_ID", how="left")
            logging.info("Successfully merged average percent disorder into master dataframe.")
        else:
            logging.info("'Mean_Percent_Disorder' column already exists in the master dataframe.")

        # --- 2. Generate the Distribution Plot ---
        logging.info("Generating intrinsic disorder distribution plot...")

        # Prepare the data for the three groups
        all_ogs_data = df_master["Mean_Percent_Disorder"].dropna()
        rigid_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "Mean_Percent_Disorder"
        ].dropna()
        plastic_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "Mean_Percent_Disorder"
        ].dropna()

        hist_data = [all_ogs_data, rigid_ogs_data, plastic_ogs_data]
        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]

        # Define colors for the distributions
        colors = [apc.slate, apc.aegean, apc.rose]

        # Create the distribution plot
        fig = ff.create_distplot(
            hist_data, group_labels, bin_size=0.01, show_hist=False, show_rug=False, colors=colors
        )

        # --- 3. Style and Annotate the Plot ---
        fig.update_layout(
            title="Distribution of Mean Intrinsic Disorder per OG",
            xaxis_title="Mean Percent Disorder",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )

        # Add a vertical line for the mean of all OGs
        mean_all = all_ogs_data.mean()
        fig.add_vline(
            x=mean_all,
            line_width=2,
            line_dash="dash",
            line_color=apc.charcoal,
            annotation_text=f"Mean (All): {mean_all:.2f}",
            annotation_position="top right",
        )

        fig.show()

        # --- 4. Export the Figure ---
        plot_filename_base = "fig3b_disorder_dist_plot"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")
        logging.info(f"Saved disorder distribution plot to {PLOTS_DIR}")

    except FileNotFoundError:
        logging.error(
            f"Proteome database not found at {PROTEOME_DB_FILE}. Cannot generate disorder plot."
        )
    except Exception as e:
        logging.error(
            f"An error occurred while creating the disorder distribution plot: {e}", exc_info=True
        )

In [ ]:
# In-Depth Correlation Analysis (Corrected)
# -------------------------------------------------
# This cell performs a multi-faceted analysis to rigorously test the
# relationship between sequence and structural diversity. It implements three
# approaches:
# 1. Global correlation with points colored by their residual (distance from the trend line).
# 2. Domain enrichment analysis of the OGs that are the biggest outliers from the global trend.
# 3. Faceted scatter plots to visualize the correlation within individual domain families.

import statsmodels.api as sm
from scipy.stats import pearsonr  # <-- CORRECTED: Added fisher_exact

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
else:
    try:
        # --- 1. Load and Prepare IPR Domain Data ---
        if "Dominant_IPR" not in df_master.columns:
            logging.info("Loading and processing IPR domain data...")
            proteome_cols = ["OG_ID", "IPR_Signatures"]
            df_ipr = pd.read_csv(PROTEOME_DB_FILE, usecols=proteome_cols, low_memory=False)
            df_ipr.dropna(subset=["OG_ID", "IPR_Signatures"], inplace=True)
            df_ipr["IPR_Signatures"] = df_ipr["IPR_Signatures"].str.split(";")
            df_ipr_long = df_ipr.explode("IPR_Signatures").rename(
                columns={"IPR_Signatures": "IPR_ID"}
            )
            df_ipr_long.dropna(subset=["IPR_ID"], inplace=True)

            dominant_ipr = (
                df_ipr_long.groupby("OG_ID")["IPR_ID"]
                .agg(lambda x: x.mode()[0] if not x.mode().empty else None)
                .reset_index()
            )
            dominant_ipr.rename(columns={"IPR_ID": "Dominant_IPR"}, inplace=True)
            df_master = pd.merge(df_master, dominant_ipr, on="OG_ID", how="left")

            # Add domain names for readability
            df_ipr_names = pd.read_csv(
                DATA_DIR / "reference/interpro_entry.txt",
                sep="\\t",
                names=["IPR_ID", "Type", "Domain_Name"],
            )
            ipr_to_name_map = pd.Series(
                df_ipr_names.Domain_Name.values, index=df_ipr_names.IPR_ID
            ).to_dict()
            df_master["Dominant_IPR_Name"] = (
                df_master["Dominant_IPR"].map(ipr_to_name_map).fillna("Unknown")
            )
            logging.info("Dominant IPR domains have been merged into the master dataframe.")

        # --- Approach 1: Global Correlation with Residuals ---
        logging.info("--- Generating Plot 1: Global Correlation Colored by Residuals ---")

        df_plot = df_master.dropna(subset=["Tree_Hill_Diversity_q1_NormByTips", "StdDev_TMscore"])
        X = sm.add_constant(df_plot["Tree_Hill_Diversity_q1_NormByTips"])
        y = df_plot["StdDev_TMscore"]
        model = sm.OLS(y, X).fit()
        df_plot["Residual"] = model.resid

        pearson_r, p_val = pearsonr(
            df_plot["Tree_Hill_Diversity_q1_NormByTips"], df_plot["StdDev_TMscore"]
        )

        fig1 = px.scatter(
            df_plot,
            x="Tree_Hill_Diversity_q1_NormByTips",
            y="StdDev_TMscore",
            color="Residual",
            color_continuous_scale="Plasma_r",
            hover_name="OG_ID",
            hover_data=["Dominant_IPR_Name", "Residual"],
            title=f"Global Correlation (R={pearson_r:.2f}, p={p_val:.2g}) Colored by Residuals",
            labels={
                "Tree_Hill_Diversity_q1_NormByTips": "Sequence Diversity",
                "StdDev_TMscore": "Structural Diversity",
            },
        )
        # Add the regression line
        x_range = np.linspace(
            df_plot["Tree_Hill_Diversity_q1_NormByTips"].min(),
            df_plot["Tree_Hill_Diversity_q1_NormByTips"].max(),
            100,
        )
        y_range = model.predict(sm.add_constant(x_range))
        fig1.add_trace(
            go.Scatter(
                x=x_range,
                y=y_range,
                mode="lines",
                name="Global Trend",
                line=dict(color=apc.charcoal, width=3, dash="dash"),
            )
        )

        fig1.update_layout(width=800, height=600)
        fig1.show()

        # --- Approach 3: Faceted Correlation by Domain Family ---
        logging.info("--- Generating Plot 3: Faceted Correlation by Domain Family ---")

        # Find the most common domain families to plot
        common_domains = df_master["Dominant_IPR_Name"].value_counts().nlargest(9).index.tolist()
        df_plot_faceted = df_master[df_master["Dominant_IPR_Name"].isin(common_domains)]

        fig3 = px.scatter(
            df_plot_faceted,
            x="Tree_Hill_Diversity_q1_NormByTips",
            y="StdDev_TMscore",
            facet_col="Dominant_IPR_Name",
            facet_col_wrap=3,
            trendline="ols",
            trendline_color_override=apc.amber,
            title="Sequence vs. Structural Diversity by Domain Family",
            labels={
                "Tree_Hill_Diversity_q1_NormByTips": "Seq. Diversity",
                "StdDev_TMscore": "Struc. Diversity",
            },
        )
        fig3.update_traces(marker=dict(color=apc.aegean))
        fig3.for_each_annotation(
            lambda a: a.update(text=a.text.split("=")[1])
        )  # Clean up facet titles
        fig3.show()

        # --- Export All Figures ---
        fig1.write_image(PLOTS_DIR / "fig4a_global_correlation_residuals.svg")
        if "df_sig_enrichment" in locals() and not df_sig_enrichment.empty:
            df_sig_enrichment.to_csv(PLOTS_DIR / "table_outlier_domain_enrichment.csv", index=False)
        fig3.write_image(PLOTS_DIR / "fig4b_faceted_correlation_by_domain.svg")
        logging.info(f"Correlation analysis plots and tables saved to {PLOTS_DIR}")

    except Exception as e:
        logging.error(f"An error occurred during correlation analysis: {e}", exc_info=True)

In [ ]:
# APSI vs. Structural Diversity with Full Statistical Analysis
# --------------------------------------------------------------------
# This cell plots structural diversity against APSI and includes a robust
# statistical comparison, reporting both the p-value and the effect size
# (Cohen's d) to quantify the magnitude of the difference.

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "APSI_Ungapped" not in df_master.columns:
    logging.error(
        "'APSI_Ungapped' column not found in df_master. Please ensure it was loaded correctly."
    )
else:
    try:
        logging.info(
            "Generating scatter plot of APSI vs. Structural Diversity with full statistical annotation..."
        )

        # --- 1. Perform Full Statistical Test on APSI ---
        rigid_apsi = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "APSI_Ungapped"
        ].dropna()
        plastic_apsi = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "APSI_Ungapped"
        ].dropna()

        # Welch's t-test for p-value
        ttest_result = stats.ttest_ind(rigid_apsi, plastic_apsi, equal_var=False)

        # Calculate Cohen's d for effect size
        mean_diff = rigid_apsi.mean() - plastic_apsi.mean()
        n1, n2 = len(rigid_apsi), len(plastic_apsi)
        var1, var2 = rigid_apsi.var(ddof=1), plastic_apsi.var(ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        cohens_d = mean_diff / pooled_std

        # --- Print detailed stats to console ---
        print("--- APSI Comparison: Rigid vs. Plastic ---")
        print(
            f"Rigid Group:     Mean APSI = {rigid_apsi.mean():.2f}%, Std Dev = {rigid_apsi.std():.2f}, n = {n1}"
        )
        print(
            f"Plastic Group:   Mean APSI = {plastic_apsi.mean():.2f}%, Std Dev = {plastic_apsi.std():.2f}, n = {n2}"
        )
        print(f"p-value: {ttest_result.pvalue:.2e}")
        print(f"Cohen's d (Effect Size): {cohens_d:.3f}")
        print("------------------------------------------")

        # --- 2. Create the Scatter Plot ---
        fig = px.scatter(
            df_master,
            x="APSI_Ungapped",
            y="StdDev_TMscore",
            color="Structural_Profile",
            color_discrete_map={
                "Structurally Rigid": apc.aegean,
                "Structurally Plastic": apc.rose,
                "Consistently Dissimilar": apc.gray,
                "Intermediate": apc.gray,
            },
            size="Marker_Size",
            hover_name="OG_ID",
            hover_data=["Mean_TMscore", "StdDev_TMscore", "MSA_N_Seqs"],
            title="Structural Diversity vs. Average Pairwise Sequence Identity",
            labels={
                "APSI_Ungapped": "Sequence Similarity (Average Pairwise Identity %)",
                "StdDev_TMscore": "Structural Diversity (StdDev TM-score)",
            },
            category_orders={
                "Structural_Profile": [
                    "Structurally Rigid",
                    "Structurally Plastic",
                    "Consistently Dissimilar",
                    "Intermediate",
                ]
            },
        )

        # Add the regression line
        x_range = np.linspace(df_plot["APSI_Ungapped"].min(), df_plot["APSI_Ungapped"].max(), 100)
        y_range = model.predict(sm.add_constant(x_range))
        fig.add_trace(
            go.Scatter(
                x=x_range,
                y=y_range,
                mode="lines",
                name="Global Trend",
                line=dict(color=apc.charcoal, width=3, dash="dash"),
            )
        )

        fig.update_xaxes(autorange="reversed")
        fig.update_layout(width=800, height=600)
        fig.show()

        # --- 3. Refine Plot Aesthetics & Add Annotation ---
        fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color="DarkSlateGrey")))
        for trace in fig.data:
            if trace.name in ["Consistently Dissimilar", "Intermediate"]:
                trace.marker.opacity = 0.15
                trace.marker.symbol = "circle"

        fig.update_xaxes(autorange="reversed")

        fig.add_annotation(
            x=0.95,
            y=0.95,
            xref="paper",
            yref="paper",
            text=f"<b>APSI Comparison (Rigid vs. Plastic):</b><br>p = {ttest_result.pvalue:.2e}<br>Cohen's d = {cohens_d:.3f}",
            showarrow=False,
            align="right",
            font=dict(size=14),
            bgcolor="rgba(255,255,255,0.7)",
        )

        fig.update_layout(width=900, height=700, legend_title_text="<b>Structural Profile</b>")
        fig.show()

        # --- 4. Calculate global Pearson correlation ---
        pearson_r, p_val = pearsonr(df_plot["APSI_Ungapped"], df_plot["StdDev_TMscore"])

        # --- 5. Export the Figure and Stats ---
        plot_filename_base = "fig_sup_apsi_vs_struc_div_with_stats"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")

        # Save detailed stats to a file
        stats_df = pd.DataFrame(
            {
                "Group": ["Structurally Rigid", "Structurally Plastic"],
                "Mean_APSI": [rigid_apsi.mean(), plastic_apsi.mean()],
                "Std_Dev_APSI": [rigid_apsi.std(), plastic_apsi.std()],
                "N": [n1, n2],
            }
        )
        stats_df.to_csv(PLOTS_DIR / "table_apsi_comparison_stats.csv", index=False)

        logging.info(f"Saved APSI vs Structural Diversity plot and stats to {PLOTS_DIR}")

    except Exception as e:
        logging.error(f"An error occurred while creating the APSI plot: {e}", exc_info=True)

In [ ]:
# Mean Shannon Entropy vs. Structural Diversity
# -----------------------------------------------------
# This cell creates the global correlation plot for mean Shannon Entropy vs.
# structural diversity, now correctly showing a single global trendline.

import statsmodels.api as sm
from scipy.stats import pearsonr

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "MSA_Mean_Col_Entropy" not in df_master.columns:
    logging.error("'MSA_Mean_Col_Entropy' column not found. Please ensure it was loaded correctly.")
else:
    try:
        logging.info("Generating scatter plot of Mean SE vs. Structural Diversity...")

        # --- 1. Prepare data and calculate global correlation ---
        df_plot = df_master.dropna(subset=["MSA_Mean_Col_Entropy", "StdDev_TMscore"]).copy()
        pearson_r, p_val = pearsonr(df_plot["MSA_Mean_Col_Entropy"], df_plot["StdDev_TMscore"])

        # --- 2. Create the Scatter Plot (without automatic trendline) ---
        fig = px.scatter(
            df_plot,
            x="MSA_Mean_Col_Entropy",
            y="StdDev_TMscore",
            color="Structural_Profile",
            color_discrete_map={
                "Structurally Rigid": apc.aegean,
                "Structurally Plastic": apc.rose,
                "Consistently Dissimilar": apc.gray,
                "Intermediate": apc.gray,
            },
            size="Marker_Size",
            hover_name="OG_ID",
            title=f"Structural Diversity vs. Mean Sequence Variability (R={pearson_r:.2f})",
            labels={
                "MSA_Mean_Col_Entropy": "Mean Per-Column Shannon Entropy",
                "StdDev_TMscore": "Structural Diversity (StdDev TM-score)",
            },
        )

        # --- 3. Manually calculate and add the single global trendline ---
        X = sm.add_constant(df_plot["MSA_Mean_Col_Entropy"])
        y = df_plot["StdDev_TMscore"]
        model = sm.OLS(y, X).fit()
        x_range = np.linspace(X["MSA_Mean_Col_Entropy"].min(), X["MSA_Mean_Col_Entropy"].max(), 100)
        y_range = model.predict(sm.add_constant(x_range))
        fig.add_trace(
            go.Scatter(
                x=x_range,
                y=y_range,
                mode="lines",
                name="Global Trend",
                line=dict(color=apc.charcoal, width=3, dash="dash"),
            )
        )

        # --- 4. Refine Plot Aesthetics ---
        fig.update_traces(marker=dict(opacity=0.7, line=dict(width=1, color="DarkSlateGrey")))
        for trace in fig.data:
            if trace.name in ["Consistently Dissimilar", "Intermediate"]:
                trace.marker.opacity = 0.15
                trace.marker.symbol = "circle"

        fig.update_layout(width=800, height=600, legend_title_text="<b>Structural Profile</b>")
        fig.show()

        # --- 5. Export the Figure ---
        plot_filename_base = "fig_sup_mean_se_vs_struc_div"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")
        logging.info(f"Saved Mean SE vs Structural Diversity plot to {PLOTS_DIR}")

    except Exception as e:
        logging.error(f"An error occurred while creating the Mean SE plot: {e}", exc_info=True)

In [ ]:
#  Mean Shannon Entropy Distribution Analysis
# ---------------------------------------------------
# This cell formally compares the distribution of the mean per-column
# Shannon Entropy (SE) across the different structural profiles. It provides
# a statistical test and a visualization to complement the scatter plot from Cell 13.

import plotly.figure_factory as ff
from scipy.stats import ttest_ind

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "MSA_Mean_Col_Entropy" not in df_master.columns:
    logging.error("'MSA_Mean_Col_Entropy' column not found. Please ensure it was loaded correctly.")
else:
    try:
        logging.info("--- Analyzing Mean Shannon Entropy Distributions ---")

        # --- 1. Prepare Data for Comparison ---
        all_ogs_data = df_master["MSA_Mean_Col_Entropy"].dropna()
        rigid_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "MSA_Mean_Col_Entropy"
        ].dropna()
        plastic_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "MSA_Mean_Col_Entropy"
        ].dropna()

        # --- 2. Perform Full Statistical Test ---
        ttest_result = ttest_ind(rigid_ogs_data, plastic_ogs_data, equal_var=False)

        mean_diff = rigid_ogs_data.mean() - plastic_ogs_data.mean()
        n1, n2 = len(rigid_ogs_data), len(plastic_ogs_data)
        var1, var2 = rigid_ogs_data.var(ddof=1), plastic_ogs_data.var(ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        cohens_d = mean_diff / pooled_std

        print("--- Mean Shannon Entropy Comparison: Rigid vs. Plastic ---")
        print(
            f"Rigid Group:     Mean SE = {rigid_ogs_data.mean():.2f}, Std Dev = {rigid_ogs_data.std():.2f}, n = {n1}"
        )
        print(
            f"Plastic Group:   Mean SE = {plastic_ogs_data.mean():.2f}, Std Dev = {plastic_ogs_data.std():.2f}, n = {n2}"
        )
        print(f"p-value: {ttest_result.pvalue:.2e}")
        print(f"Cohen's d (Effect Size): {cohens_d:.3f}")
        print("----------------------------------------------------------")

        # --- 3. Generate the Distribution Plot ---
        hist_data = [all_ogs_data, rigid_ogs_data, plastic_ogs_data]
        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]
        colors = [apc.slate, apc.aegean, apc.rose]

        fig = ff.create_distplot(
            hist_data, group_labels, bin_size=0.05, show_hist=False, show_rug=False, colors=colors
        )

        # --- 4. Style and Annotate the Plot ---
        fig.update_layout(
            title="Distribution of Mean Shannon Entropy per OG",
            xaxis_title="Mean Per-Column Shannon Entropy (Variability)",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )

        mean_all = all_ogs_data.mean()
        fig.add_vline(
            x=mean_all,
            line_width=2,
            line_dash="dash",
            line_color=apc.charcoal,
            annotation_text=f"Mean (All): {mean_all:.2f}",
            annotation_position="top right",
        )

        fig.show()

        # --- 5. Export the Figure and Stats ---
        plot_filename_base = "fig_sup_mean_se_distribution"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")

        stats_df = pd.DataFrame(
            {
                "Group": ["Structurally Rigid", "Structurally Plastic"],
                "Mean_SE": [rigid_ogs_data.mean(), plastic_ogs_data.mean()],
                "Std_Dev_SE": [rigid_ogs_data.std(), plastic_ogs_data.std()],
                "N": [n1, n2],
            }
        )
        stats_df.to_csv(PLOTS_DIR / "table_mean_se_comparison_stats.csv", index=False)

        logging.info(f"Saved Mean SE distribution plot and stats to {PLOTS_DIR}")

    except Exception as e:
        logging.error(
            f"An error occurred while creating the Mean SE distribution plot: {e}", exc_info=True
        )

In [ ]:
# Distribution of Mean and Median TM-scores with Full Statistical Analysis
# --------------------------------------------------------------------------------
# This cell visualizes the distribution of the average structural similarity
# and includes formal statistical tests and descriptive stats for all groups.

import plotly.figure_factory as ff
from scipy.stats import mannwhitneyu, ttest_ind

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "Mean_TMscore" not in df_master.columns or "Median_TMscore" not in df_master.columns:
    logging.error(
        "'Mean_TMscore' or 'Median_TMscore' columns not found. Please ensure they were loaded correctly."
    )
else:
    try:
        logging.info("--- Analyzing Mean and Median TM-score Distributions ---")

        # --- 1. Prepare Data for Plotting and Stats ---
        all_mean_tm = df_master["Mean_TMscore"].dropna()
        rigid_mean_tm = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "Mean_TMscore"
        ].dropna()
        plastic_mean_tm = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "Mean_TMscore"
        ].dropna()
        hist_data_mean = [all_mean_tm, rigid_mean_tm, plastic_mean_tm]

        all_median_tm = df_master["Median_TMscore"].dropna()
        rigid_median_tm = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "Median_TMscore"
        ].dropna()
        plastic_median_tm = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "Median_TMscore"
        ].dropna()
        hist_data_median = [all_median_tm, rigid_median_tm, plastic_median_tm]

        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]
        colors = [apc.slate, apc.aegean, apc.rose]

        # --- 2. Statistical Analysis for Mean TM-score ---
        ttest_mean = ttest_ind(rigid_mean_tm, plastic_mean_tm, equal_var=False)
        mean_diff = rigid_mean_tm.mean() - plastic_mean_tm.mean()
        n1, n2 = len(rigid_mean_tm), len(plastic_mean_tm)
        var1, var2 = rigid_mean_tm.var(ddof=1), plastic_mean_tm.var(ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        cohens_d_mean = mean_diff / pooled_std

        print("--- Mean TM-score Comparison ---")
        print(
            f"All OGs:       Mean={all_mean_tm.mean():.3f}, StdDev={all_mean_tm.std():.3f}, n={len(all_mean_tm)}"
        )
        print(
            f"Rigid Group:   Mean={rigid_mean_tm.mean():.3f}, StdDev={rigid_mean_tm.std():.3f}, n={n1}"
        )
        print(
            f"Plastic Group: Mean={plastic_mean_tm.mean():.3f}, StdDev={plastic_mean_tm.std():.3f}, n={n2}"
        )
        print(
            f"\nRigid vs. Plastic: p-value = {ttest_mean.pvalue:.2e}, Cohen's d = {cohens_d_mean:.3f}"
        )
        print("---------------------------------")

        # --- 3. Generate KDE Plot for Mean TM-score ---
        fig_mean = ff.create_distplot(
            hist_data_mean,
            group_labels,
            colors=colors,
            bin_size=0.02,
            show_hist=False,
            show_rug=False,
        )
        fig_mean.update_layout(
            title="Distribution of Mean Pairwise TM-score per OG",
            xaxis_title="Mean TM-score",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )
        print("\n--- Plot 1: Distribution of Mean TM-score ---")
        fig_mean.show()

        # --- 4. Statistical Analysis for Median TM-score ---
        u_stat, p_val_median = mannwhitneyu(
            rigid_median_tm, plastic_median_tm, alternative="two-sided"
        )

        print("\n--- Median TM-score Comparison ---")
        print(f"All OGs:       Median={all_median_tm.median():.3f}, n={len(all_median_tm)}")
        print(f"Rigid Group:   Median={rigid_median_tm.median():.3f}, n={len(rigid_median_tm)}")
        print(f"Plastic Group: Median={plastic_median_tm.median():.3f}, n={len(plastic_median_tm)}")
        print(f"\nRigid vs. Plastic (Mann-Whitney U test): p-value = {p_val_median:.2e}")
        print("-----------------------------------")

        # --- 5. Generate KDE Plot for Median TM-score ---
        fig_median = ff.create_distplot(
            hist_data_median,
            group_labels,
            colors=colors,
            bin_size=0.02,
            show_hist=False,
            show_rug=False,
        )
        fig_median.update_layout(
            title="Distribution of Median Pairwise TM-score per OG",
            xaxis_title="Median TM-score",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )
        print("\n--- Plot 2: Distribution of Median TM-score ---")
        fig_median.show()

        # --- 6. Export Figures ---
        fig_mean.write_image(PLOTS_DIR / "fig_sup_mean_tmscore_dist.svg")
        fig_median.write_image(PLOTS_DIR / "fig_sup_median_tmscore_dist.svg")
        logging.info(f"Saved TM-score distribution plots to {PLOTS_DIR}")

    except Exception as e:
        logging.error(
            f"An error occurred while creating the TM-score distribution plots: {e}", exc_info=True
        )

In [ ]:
# Domain Architecture Distribution Plot with Statistical Analysis
# ----------------------------------------------------------------------
# This cell loads domain data, generates the KDE plot, and now includes
# a full statistical comparison of the mean domain counts between groups.

import plotly.figure_factory as ff
from scipy.stats import ttest_ind

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run the previous cells first."
    )
elif "DATA_DIR" not in locals() or "PLOTS_DIR" not in locals():
    logging.error(
        "Path variables 'DATA_DIR' or 'PLOTS_DIR' are not defined. Please run the setup cell (Cell 2)."
    )
else:
    try:
        # --- 1. Load and Prepare Domain Data ---
        if "Mean_Num_Domains" not in df_master.columns:
            logging.info("Loading domain information from the main proteome database...")
            proteome_db_path = DATA_DIR / "proteome_database_v3.5.csv"
            proteome_cols = ["OG_ID", "Num_Domains"]

            df_proteome_domains = pd.read_csv(
                proteome_db_path, usecols=proteome_cols, low_memory=False
            )

            df_proteome_domains["Num_Domains"] = pd.to_numeric(
                df_proteome_domains["Num_Domains"], errors="coerce"
            )
            df_proteome_domains.dropna(subset=["OG_ID", "Num_Domains"], inplace=True)

            logging.info("Calculating average number of domains per OG...")
            df_og_domain_avg = (
                df_proteome_domains.groupby("OG_ID")["Num_Domains"].mean().reset_index()
            )
            df_og_domain_avg = df_og_domain_avg.rename(columns={"Num_Domains": "Mean_Num_Domains"})

            df_master = pd.merge(df_master, df_og_domain_avg, on="OG_ID", how="left")
            logging.info("Successfully merged average domain counts into master dataframe.")
        else:
            logging.info("'Mean_Num_Domains' column already exists in the master dataframe.")

        # --- 2. Perform Statistical Analysis ---
        all_ogs_data = df_master["Mean_Num_Domains"].dropna()
        rigid_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "Mean_Num_Domains"
        ].dropna()
        plastic_ogs_data = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "Mean_Num_Domains"
        ].dropna()

        ttest_result = ttest_ind(rigid_ogs_data, plastic_ogs_data, equal_var=False)
        mean_diff = plastic_ogs_data.mean() - rigid_ogs_data.mean()
        n1, n2 = len(rigid_ogs_data), len(plastic_ogs_data)
        var1, var2 = rigid_ogs_data.var(ddof=1), plastic_ogs_data.var(ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        cohens_d = mean_diff / pooled_std

        print("--- Mean Domain Count Comparison ---")
        print(
            f"All OGs:       Mean={all_ogs_data.mean():.2f}, StdDev={all_ogs_data.std():.2f}, n={len(all_ogs_data)}"
        )
        print(
            f"Rigid Group:   Mean={rigid_ogs_data.mean():.2f}, StdDev={rigid_ogs_data.std():.2f}, n={n1}"
        )
        print(
            f"Plastic Group: Mean={plastic_ogs_data.mean():.2f}, StdDev={plastic_ogs_data.std():.2f}, n={n2}"
        )
        print(
            f"\nRigid vs. Plastic: p-value = {ttest_result.pvalue:.2e}, Cohen's d = {cohens_d:.3f}"
        )
        print("------------------------------------")

        # --- 3. Generate the Distribution Plot ---
        hist_data = [all_ogs_data, rigid_ogs_data, plastic_ogs_data]
        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]
        colors = [apc.slate, apc.aegean, apc.rose]

        fig = ff.create_distplot(
            hist_data, group_labels, bin_size=0.2, show_hist=False, show_rug=False, colors=colors
        )

        fig.update_layout(
            title="Distribution of Mean Domain Count per OG",
            xaxis_title="Mean Number of Domains",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )

        mean_all = all_ogs_data.mean()
        fig.add_vline(
            x=mean_all,
            line_width=2,
            line_dash="dash",
            line_color=apc.charcoal,
            annotation_text=f"Mean (All): {mean_all:.2f}",
            annotation_position="top right",
        )

        print("\n--- Plot: Distribution of Mean Domain Count ---")
        fig.show()

        # --- 4. Export the Figure ---
        plot_filename_base = "fig3a_domain_dist_plot"
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")
        logging.info(f"Saved domain distribution plot to {PLOTS_DIR}")

    except FileNotFoundError:
        logging.error("Proteome database not found. Cannot generate domain plot.")
    except Exception as e:
        logging.error(f"An error occurred while creating the distribution plot: {e}", exc_info=True)

In [ ]:
# APSI Distribution Analysis
# ----------------------------------
# This cell creates a distribution plot (KDE) for the Average Pairwise
# Sequence Identity (APSI) to visually complement the scatter plot.
# It includes a full statistical comparison of the APSI values
# between the 'Rigid' and 'Plastic' groups.

import plotly.figure_factory as ff
from scipy.stats import ttest_ind

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "APSI_Ungapped" not in df_master.columns:
    logging.error("'APSI_Ungapped' column not found. Please ensure it was loaded correctly.")
else:
    try:
        logging.info("--- Analyzing APSI Distributions ---")

        # --- 1. Prepare Data for Plotting and Stats ---
        all_apsi = df_master["APSI_Ungapped"].dropna()
        rigid_apsi = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "APSI_Ungapped"
        ].dropna()
        plastic_apsi = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "APSI_Ungapped"
        ].dropna()

        hist_data = [all_apsi, rigid_apsi, plastic_apsi]
        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]
        colors = [apc.slate, apc.aegean, apc.rose]

        # --- 2. Perform Full Statistical Test (as in Cell 9) ---
        ttest_result = ttest_ind(rigid_apsi, plastic_apsi, equal_var=False)

        mean_diff = rigid_apsi.mean() - plastic_apsi.mean()
        n1, n2 = len(rigid_apsi), len(plastic_apsi)
        var1, var2 = rigid_apsi.var(ddof=1), plastic_apsi.var(ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        cohens_d = mean_diff / pooled_std

        print("--- APSI Comparison: Rigid vs. Plastic ---")
        print(f"All OGs:       Mean APSI = {all_apsi.mean():.2f}%, n={len(all_apsi)}")
        print(f"Rigid Group:   Mean APSI = {rigid_apsi.mean():.2f}%, n={n1}")
        print(f"Plastic Group: Mean APSI = {plastic_apsi.mean():.2f}%, n={n2}")
        print(
            f"\nRigid vs. Plastic: p-value = {ttest_result.pvalue:.2e}, Cohen's d = {cohens_d:.3f}"
        )
        print("------------------------------------------")

        # --- 3. Generate the Distribution Plot ---
        fig = ff.create_distplot(
            hist_data, group_labels, colors=colors, bin_size=1.0, show_hist=False, show_rug=False
        )

        # --- 4. Style and Annotate the Plot ---
        fig.update_layout(
            title="Distribution of Average Pairwise Sequence Identity (APSI)",
            xaxis_title="Average Pairwise Identity (%)",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )

        fig.show()

        # --- 5. Export the Figure ---
        plot_filename_base = "fig_sup_apsi_distribution"
        fig.write_html(PLOTS_DIR / f"{plot_filename_base}.html")
        fig.write_image(PLOTS_DIR / f"{plot_filename_base}.svg")
        logging.info(f"Saved APSI distribution plot to {PLOTS_DIR}")

    except Exception as e:
        logging.error(
            f"An error occurred while creating the APSI distribution plot: {e}", exc_info=True
        )

In [ ]:
# Quantifying High-Diversity Rigid OGs
# ---------------------------------------------
# This cell quantifies the number and percentage of "Structurally Rigid" OGs
# that exhibit high sequence diversity. It uses multiple definitions for
# "high diversity" for a comprehensive comparison.
# The results are printed and saved to a CSV file.

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "APSI_Ungapped" not in df_master.columns or "MSA_Mean_Col_Entropy" not in df_master.columns:
    logging.error(
        "Required sequence diversity columns are not present. Please ensure they were loaded correctly."
    )
else:
    try:
        logging.info("--- Quantifying the 'High-Diversity Rigid' Population ---")

        # --- 1. Isolate the relevant data subsets ---
        df_rigid = df_master[df_master["Structural_Profile"] == "Structurally Rigid"].copy()
        df_plastic = df_master[df_master["Structural_Profile"] == "Structurally Plastic"].copy()
        total_rigid_ogs = len(df_rigid)

        # --- 2. Define All Thresholds ---
        # Thresholds based on quartiles of the ENTIRE dataset
        apsi_quartile_threshold = df_master["APSI_Ungapped"].quantile(0.25)
        se_quartile_threshold = df_master["MSA_Mean_Col_Entropy"].quantile(0.75)

        # Thresholds based on the MEDIAN of different groups
        apsi_plastic_median = df_plastic["APSI_Ungapped"].median()
        se_plastic_median = df_plastic["MSA_Mean_Col_Entropy"].median()
        apsi_all_median = df_master["APSI_Ungapped"].median()
        se_all_median = df_master["MSA_Mean_Col_Entropy"].median()

        # Thresholds based on the MEAN of different groups
        apsi_plastic_mean = df_plastic["APSI_Ungapped"].mean()
        se_plastic_mean = df_plastic["MSA_Mean_Col_Entropy"].mean()
        apsi_all_mean = df_master["APSI_Ungapped"].mean()
        se_all_mean = df_master["MSA_Mean_Col_Entropy"].mean()

        # --- 3. Perform all calculations ---
        # By APSI
        count_apsi_q1 = len(df_rigid[df_rigid["APSI_Ungapped"] <= apsi_quartile_threshold])
        count_apsi_plastic_median = len(df_rigid[df_rigid["APSI_Ungapped"] <= apsi_plastic_median])
        count_apsi_all_median = len(df_rigid[df_rigid["APSI_Ungapped"] <= apsi_all_median])
        count_apsi_plastic_mean = len(df_rigid[df_rigid["APSI_Ungapped"] <= apsi_plastic_mean])
        count_apsi_all_mean = len(df_rigid[df_rigid["APSI_Ungapped"] <= apsi_all_mean])

        # By Mean SE
        count_se_q3 = len(df_rigid[df_rigid["MSA_Mean_Col_Entropy"] >= se_quartile_threshold])
        count_se_plastic_median = len(
            df_rigid[df_rigid["MSA_Mean_Col_Entropy"] >= se_plastic_median]
        )
        count_se_all_median = len(df_rigid[df_rigid["MSA_Mean_Col_Entropy"] >= se_all_median])
        count_se_plastic_mean = len(df_rigid[df_rigid["MSA_Mean_Col_Entropy"] >= se_plastic_mean])
        count_se_all_mean = len(df_rigid[df_rigid["MSA_Mean_Col_Entropy"] >= se_all_mean])

        # --- 4. Print and Save a clear summary ---
        print("\n--- Analysis of 'Structurally Rigid' OGs with High Sequence Diversity ---")
        print(f"Total number of 'Structurally Rigid' OGs: {total_rigid_ogs}")

        results_data = {
            "Comparison Metric": [
                "APSI",
                "Mean SE",
                "APSI",
                "Mean SE",
                "APSI",
                "Mean SE",
                "APSI",
                "Mean SE",
            ],
            "Diversity Threshold": [
                f"<= {apsi_quartile_threshold:.2f}% (Bottom 25% of all OGs)",
                f">= {se_quartile_threshold:.2f} (Top 25% of all OGs)",
                f"<= {apsi_plastic_median:.2f}% (Median of Plastic OGs)",
                f">= {se_plastic_median:.2f} (Median of Plastic OGs)",
                f"<= {apsi_all_median:.2f}% (Median of All OGs)",
                f">= {se_all_median:.2f} (Median of All OGs)",
                f"<= {apsi_plastic_mean:.2f}% (Mean of Plastic OGs)",
                f">= {se_plastic_mean:.2f} (Mean of Plastic OGs)",
            ],
            "Count of Rigid OGs": [
                count_apsi_q1,
                count_se_q3,
                count_apsi_plastic_median,
                count_se_plastic_median,
                count_apsi_all_median,
                count_se_all_median,
                count_apsi_plastic_mean,
                count_se_plastic_mean,
            ],
            "Percentage of Rigid OGs": [
                (c / total_rigid_ogs) * 100 if total_rigid_ogs > 0 else 0
                for c in [
                    count_apsi_q1,
                    count_se_q3,
                    count_apsi_plastic_median,
                    count_se_plastic_median,
                    count_apsi_all_median,
                    count_se_all_median,
                    count_apsi_plastic_mean,
                    count_se_plastic_mean,
                ]
            ],
        }
        df_results = pd.DataFrame(results_data)

        print("\nResults:")
        display(df_results)
        print("----------------------------------------------------------------------")

        output_path = PLOTS_DIR / "table_high_diversity_rigid_quantification.csv"
        df_results.to_csv(output_path, index=False, float_format="%.2f")
        logging.info(f"Saved quantification results to: {output_path}")

    except Exception as e:
        logging.error(f"An error occurred during the analysis: {e}", exc_info=True)

In [ ]:
# Characterizing High-Diversity Rigid OGs
# ------------------------------------------------
# This cell identifies the specific OGs that are "Structurally Rigid" yet
# exhibit high sequence diversity (as defined by the mean of the plastic group).
# It then analyzes the functional annotations (Dominant IPR) of these groups
# to search for common themes.

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "Dominant_IPR_Name" not in df_master.columns:
    logging.error(
        "'Dominant_IPR_Name' column not found. Please ensure the correlation analysis cell has been run to generate it."
    )
else:
    try:
        logging.info("--- Characterizing the 'High-Diversity Rigid' Subsets ---")

        # --- 1. Isolate the relevant data subsets ---
        df_rigid = df_master[df_master["Structural_Profile"] == "Structurally Rigid"].copy()
        df_plastic = df_master[df_master["Structural_Profile"] == "Structurally Plastic"].copy()

        # --- 2. Define the thresholds from the Cell 21 output ---
        apsi_plastic_mean_threshold = df_plastic["APSI_Ungapped"].mean()
        se_plastic_mean_threshold = df_plastic["MSA_Mean_Col_Entropy"].mean()

        # --- 3. Identify the OGs in each group of interest ---
        # Group 1: Rigid OGs with APSI <= mean of plastic OGs
        rigid_by_apsi = df_rigid[df_rigid["APSI_Ungapped"] <= apsi_plastic_mean_threshold]

        # Group 2: Rigid OGs with Mean SE >= mean of plastic OGs
        rigid_by_se = df_rigid[df_rigid["MSA_Mean_Col_Entropy"] >= se_plastic_mean_threshold]

        # --- 4. Analyze and display the most common annotations for each group ---

        print(
            f"\n--- Top Annotations for {len(rigid_by_apsi)} Rigid OGs with High Diversity (by APSI <= {apsi_plastic_mean_threshold:.2f}%) ---"
        )
        if not rigid_by_apsi.empty:
            display(rigid_by_apsi["Dominant_IPR_Name"].value_counts().nlargest(10).reset_index())
        else:
            print("No OGs found in this category.")

        print(
            f"\n--- Top Annotations for {len(rigid_by_se)} Rigid OGs with High Diversity (by Mean SE >= {se_plastic_mean_threshold:.2f}) ---"
        )
        if not rigid_by_se.empty:
            display(rigid_by_se["Dominant_IPR_Name"].value_counts().nlargest(10).reset_index())
        else:
            print("No OGs found in this category.")

        # --- Optional: Save these lists to CSV for further inspection ---
        output_path_apsi = PLOTS_DIR / "list_high_div_rigid_by_apsi.csv"
        rigid_by_apsi.to_csv(output_path_apsi, index=False)
        logging.info(f"Saved list of {len(rigid_by_apsi)} OGs to {output_path_apsi}")

        output_path_se = PLOTS_DIR / "list_high_div_rigid_by_se.csv"
        rigid_by_se.to_csv(output_path_se, index=False)
        logging.info(f"Saved list of {len(rigid_by_se)} OGs to {output_path_se}")

    except Exception as e:
        logging.error(f"An error occurred during the analysis: {e}", exc_info=True)

In [ ]:
# Normalized Hill Diversity Distribution Analysis
# --------------------------------------------------------
# This cell compares the distribution of evolutionary divergence (Normalized
# Hill Diversity) across the 'Rigid' and 'Plastic' structural profiles.

import plotly.figure_factory as ff
from scipy.stats import ttest_ind

# --- Check for necessary variables ---
if "df_master" not in locals() or df_master.empty:
    logging.error(
        "Master dataframe 'df_master' is not defined or is empty. Please run previous cells first."
    )
elif "Tree_Hill_Diversity_q1_NormByTips" not in df_master.columns:
    logging.error(
        "'Tree_Hill_Diversity_q1_NormByTips' column not found. Please ensure it was loaded correctly."
    )
else:
    try:
        logging.info("--- Analyzing Normalized Hill Diversity Distributions ---")

        # --- 1. Prepare Data for Plotting and Stats ---
        all_hill = df_master["Tree_Hill_Diversity_q1_NormByTips"].dropna()
        rigid_hill = df_master[df_master["Structural_Profile"] == "Structurally Rigid"][
            "Tree_Hill_Diversity_q1_NormByTips"
        ].dropna()
        plastic_hill = df_master[df_master["Structural_Profile"] == "Structurally Plastic"][
            "Tree_Hill_Diversity_q1_NormByTips"
        ].dropna()
        hist_data_hill = [all_hill, rigid_hill, plastic_hill]

        group_labels = ["All OGs", "Structurally Rigid", "Structurally Plastic"]
        colors = [apc.slate, apc.aegean, apc.rose]

        # --- 2. Perform Full Statistical Test ---
        ttest_hill = ttest_ind(rigid_hill, plastic_hill, equal_var=False)

        mean_diff = rigid_hill.mean() - plastic_hill.mean()
        n1, n2 = len(rigid_hill), len(plastic_hill)
        var1, var2 = rigid_hill.var(ddof=1), plastic_hill.var(ddof=1)
        pooled_std = np.sqrt(((n1 - 1) * var1 + (n2 - 1) * var2) / (n1 + n2 - 2))
        cohens_d_hill = mean_diff / pooled_std

        print("--- Normalized Hill Diversity Comparison: Rigid vs. Plastic ---")
        print(f"All OGs:       Mean Hill Diversity = {all_hill.mean():.3f}, n={len(all_hill)}")
        print(f"Rigid Group:   Mean Hill Diversity = {rigid_hill.mean():.3f}, n={n1}")
        print(f"Plastic Group: Mean Hill Diversity = {plastic_hill.mean():.3f}, n={n2}")
        print(
            f"\nRigid vs. Plastic: p-value = {ttest_hill.pvalue:.2e}, Cohen's d = {cohens_d_hill:.3f}"
        )
        print("------------------------------------------------------------")

        # --- 3. Generate KDE Plot ---
        fig = ff.create_distplot(
            hist_data_hill,
            group_labels,
            colors=colors,
            bin_size=0.02,
            show_hist=False,
            show_rug=False,
        )
        fig.update_layout(
            title="Distribution of Normalized Hill Diversity per OG",
            xaxis_title="Normalized Hill Diversity (Sequence Diversity)",
            yaxis_title="Density",
            legend_title_text="<b>OG Category</b>",
            width=800,
            height=500,
        )
        print("\n--- Plot: Distribution of Hill Diversity ---")
        fig.show()

        # --- 4. Export Figure ---
        fig.write_image(PLOTS_DIR / "fig_sup_hill_diversity_dist.svg")
        logging.info(f"Saved Hill Diversity distribution plot to {PLOTS_DIR}")

    except Exception as e:
        logging.error(
            f"An error occurred while creating the Hill Diversity distribution plot: {e}",
            exc_info=True,
        )